In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pyperclip
import time
import pandas as pd
import os
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException, StaleElementReferenceException
import re



# 전역 변수 선언
UserName = ""  # 전역 변수로 초기화

# ✅ 브라우저 창 크기 설정
WINDOW_WIDTH = 1100
WINDOW_HEIGHT = 1000



# ✅ 로그인 함수 (기존 함수 유지) https://nid.naver.com/nidlogin.login?url=https%3A%2F%2Fsection.cafe.naver.com%2Fca-fe%2Fhome
# ✅ 브라우저 초기화 및 로그인 (기존 함수 유지)
def setup_browser(position_x):
    driver = webdriver.Chrome()
    driver.set_window_size(WINDOW_WIDTH, WINDOW_HEIGHT)
    driver.set_window_position(position_x, 0)
    driver.get('https://nid.naver.com/nidlogin.login?url=https%3A%2F%2Fsection.cafe.naver.com%2Fca-fe%2Fhome')
    time.sleep(0.2)
    return driver


# ✅ 로그인 함수 (기존 함수 유지)
def login_to_console(driver):
    try:
        # 아이디 입력
        id_input = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "id")))
        pyperclip.copy("jjg1945")
        id_input.send_keys(Keys.CONTROL, 'v')
        time.sleep(0.5)

        # 비밀번호 입력
        pw_input = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "pw")))
        pyperclip.copy("rkqtnxjf7&")
        pw_input.send_keys(Keys.CONTROL, 'v')
        time.sleep(1)

        # 로그인 버튼 클릭
        login_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[type="submit"]'))
        )
        login_button.click()
        print("✅ 로그인 완료!")
        time.sleep(2)
    except Exception as e:
        print(f"⚠️ 로그인 실패: {e}")

# ✅ 콘솔 페이지 이동 (기존 함수 유지)
def navigate_to_page(driver, page):
    url = f'https://cafe.naver.com/f-e/cafes/30904299/menus/69?viewType=L&page={page}'
    driver.get(url)
    time.sleep(2)  # 로딩 대기

    try:
        driver.switch_to.default_content()  # iframe 진입 전 초기화
        driver.switch_to.frame("cafe_main")
        print(f"✅ [{page}페이지] iframe 전환 완료")
    except Exception as e:
        print(f"⚠️ iframe 전환 실패: {e}")

    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    print(f"✅ [{page}페이지] 페이지 로드 완료")


def extract_article_numbers(driver, page):
    try:
        elements = driver.find_elements(By.CSS_SELECTOR, 'td.td_normal.type_articleNumber')
        numbers = [elem.text.strip() for elem in elements if elem.text.strip().isdigit()]
        data = [{'페이지': page, '글번호': num} for num in numbers]
        print(f"✅ [{page}페이지] 글번호 {len(numbers)}개 추출 완료")
        return data
    except Exception as e:
        print(f"⚠️ 글번호 추출 실패: {e}")
        return []

import re

def extract_nickname_from_article(driver, article_id):
    url = f"https://cafe.naver.com/f-e/cafes/30904299/articles/{article_id}"
    driver.get(url)
    time.sleep(1)

    try:
        driver.switch_to.default_content()
        driver.switch_to.frame("cafe_main")
        time.sleep(1)

        spans = driver.find_elements(By.TAG_NAME, 'span')
        for span in spans:
            text = span.text.strip()
            if "인게임 닉네임:" in text:
                match = re.search(r"인게임 닉네임:\s*(.+)", text)
                if match:
                    nickname = match.group(1).strip()
                    print(f"✅ [글번호 {article_id}] 닉네임: {nickname}")
                    return nickname

        print(f"❌ [글번호 {article_id}] 닉네임 미발견")
        return None

    except Exception as e:
        print(f"⚠️ [글번호 {article_id}] 닉네임 추출 오류: {e}")
        return None




#원하는 페이지 수
pageNum = 7

if __name__ == "__main__":
    driver = setup_browser(0)
    login_to_console(driver)
    time.sleep(2)

    all_data = []

    for page in range(1, pageNum+1):
        navigate_to_page(driver, page)
        page_data = extract_article_numbers(driver, page)
        all_data.extend(page_data)
        time.sleep(0.3)

    # ✅ 닉네임 추출 시작
    enriched_data = []
    for item in all_data:
        article_id = item['글번호']
        nickname = extract_nickname_from_article(driver, article_id)
        enriched_data.append({
            '페이지': item['페이지'],
            '글번호': article_id,
            '닉네임': nickname
        })
        time.sleep(0.1)

    # ✅ 엑셀 저장
    df = pd.DataFrame(enriched_data)
    df.to_excel("리뷰글정보.xlsx", index=False)
    print("✅ 닉네임 포함 엑셀 저장 완료!")

    driver.quit()



✅ 로그인 완료!
⚠️ iframe 전환 실패: Message: cafe_main

✅ [1페이지] 페이지 로드 완료
✅ [1페이지] 글번호 15개 추출 완료
⚠️ iframe 전환 실패: Message: cafe_main

✅ [2페이지] 페이지 로드 완료
✅ [2페이지] 글번호 15개 추출 완료
⚠️ iframe 전환 실패: Message: cafe_main

✅ [3페이지] 페이지 로드 완료
✅ [3페이지] 글번호 15개 추출 완료
⚠️ iframe 전환 실패: Message: cafe_main

✅ [4페이지] 페이지 로드 완료
✅ [4페이지] 글번호 15개 추출 완료
⚠️ iframe 전환 실패: Message: cafe_main

✅ [5페이지] 페이지 로드 완료
✅ [5페이지] 글번호 15개 추출 완료
⚠️ iframe 전환 실패: Message: cafe_main

✅ [6페이지] 페이지 로드 완료
✅ [6페이지] 글번호 15개 추출 완료
⚠️ iframe 전환 실패: Message: cafe_main

✅ [7페이지] 페이지 로드 완료
✅ [7페이지] 글번호 15개 추출 완료
✅ [글번호 11076] 닉네임: 시인아이
✅ [글번호 11074] 닉네임: P용신
✅ [글번호 11066] 닉네임: 원소
✅ [글번호 11065] 닉네임: 노을
⚠️ [글번호 11055] 닉네임 추출 오류: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=139.0.7258.139)
Stacktrace:
	GetHandleVerifier [0x0x7ff6019b78d5+2802741]
	GetHandleVerifier [0x0x7ff60171eb70+79568]
	(No symbol) [0x0x7ff6014bc0fa]
	(No symbol) [0

InvalidSessionIdException: Message: invalid session id
Stacktrace:
	GetHandleVerifier [0x0x7ff6019b78d5+2802741]
	GetHandleVerifier [0x0x7ff60171eb70+79568]
	(No symbol) [0x0x7ff6014bbf2c]
	(No symbol) [0x0x7ff601502dbf]
	(No symbol) [0x0x7ff60153ab52]
	(No symbol) [0x0x7ff601535663]
	(No symbol) [0x0x7ff601534729]
	(No symbol) [0x0x7ff601486fd5]
	GetHandleVerifier [0x0x7ff6019e2ced+2979917]
	GetHandleVerifier [0x0x7ff6019dd0f3+2956371]
	GetHandleVerifier [0x0x7ff6019facbd+3078173]
	GetHandleVerifier [0x0x7ff60173836e+184014]
	GetHandleVerifier [0x0x7ff60174024f+216495]
	(No symbol) [0x0x7ff601485fd1]
	GetHandleVerifier [0x0x7ff601afe1a8+4140296]
	BaseThreadInitThunk [0x0x7ffcf9dc257d+29]
	RtlUserThreadStart [0x0x7ffcfbdaaf08+40]
